In [ ]:
# Load a spacy model and chekc if it has ner
import spacy
nlp=spacy.load('en_core_web_sm')
from spacy.tokens import Span
import difflib

In [ ]:
import numpy as np
import pandas as pd
data = pd.read_csv('task_1_information_extraction_valid.tsv', sep='\t')
print(len(data))
data = data[data['NOTES'].notna()]
data = data.reset_index(drop=True)
print(len(data))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import nltk
import spacy
  
# essential entity models downloads
nltk.downloader.download('maxent_ne_chunker')
nltk.downloader.download('words')
nltk.downloader.download('treebank')
nltk.downloader.download('maxent_treebank_pos_tagger')
nltk.downloader.download('punkt')
nltk.download('averaged_perceptron_tagger')

In [ ]:
! pip install locationtagger
import locationtagger

In [ ]:
pd.isna(data['NOTES'][40:45])

In [ ]:

import numpy as np

def getlocation(param, Locations):
    for i in range(len(data)):
        
        line = data['NOTES'][i]

        if (i % 1000) == 0 and (i != 0):
            print("Finished Sentense :", i)

        place_entity = locationtagger.find_locations(text = line)

        if param == 'countries':
            Locations1.append(place_entity.countries)
        
        if param == 'regions':
            Locations2.append(place_entity.regions)
        
        if param == 'cities':
            Locations3.append(place_entity.cities)

# Getting Locations
Locations1 = []
# Locations2 = []
# Locations3 = []

getlocation('countries', Locations1)
# getlocation('regions', Locations2)
# getlocation('cities', Locations3)

In [ ]:
print(Locations1)

In [ ]:
print(Locations2)

In [ ]:
print(Locations3)

In [ ]:
def getcounts(Locations):
    count = 0
    for i in range(len(Locations)):
        if not Locations[i]:
            count = count + 1

    print(len(Locations1))
    print("Total Empty Locations: ", count)

getcounts(Locations1)

In [ ]:
# Using Spacy on Empty Locations
def spacy(Locations):
    for i in range(len(Locations)):
        if not Locations[i]:
            doc = nlp(data['NOTES'][i])
            loc = []
            for ent in doc.ents:  
                if ent.label_ in 'LOC':
                    Locations[i] = ent.text
                    # loc.append(ent.text)
                    continue
                elif ent.label_ in 'GPE':
                    # loc.append(ent.text)
                    Locations[i] = ent.text
                    continue
spacy(Locations1)

In [ ]:
getcounts(Locations1)

In [ ]:
Actual_Locations = data['LOCATION']

In [ ]:
Locations1

In [ ]:
# Accuracy
correct = 0

for i in range(len(Locations1)):

    if not Locations1[i]:
        continue

    if difflib.SequenceMatcher(None,Locations1[i],Actual_Locations[i]).ratio() >= 0.5:
        correct += 1

print(correct)
print(correct/len(Locations1))

In [ ]:
# Getting Fatalities
Numbers = []
import numpy as np
for i in range(len(data)):
    line = data['NOTES'][i]
    doc = nlp(line)

    if (i % 1000) == 0 and (i != 0):
        print("Finished Sentense :", i)

    num = []
    for ent in doc.ents:  
        if ent.label_ in ['CARDINAL']:
            num.append(ent.text)
        else:
            num.append(0)

    Numbers.append(num)

In [ ]:
len(Numbers)

In [ ]:
# Removing sentences which return 0 cardinals and considering only the first cardinal in a sentence
F1 = []
for i in range(len(Numbers)):
    if Numbers[i] != []:
        F1.append(Numbers[i][0])
    else:
        F1.append(0)

In [ ]:
# Removing dates from sentences 
from re import search

F2 = F1.copy()
for i in range(len(F2)):
    if type(F2[i]) == str:
        if search('/', F2[i]):
            F2[i] = 0

In [ ]:
Actual_Fatalities = data['FATALITIES']

In [ ]:
# Calculating Accuracy
correct = 0
result1 = []
result2 = []
for i in range(len(Actual_Fatalities)):
    if Actual_Fatalities[i] == F2[i]:

        correct += 1
        result1.append([i, Actual_Fatalities[i], F2[i]])

print(correct)
print(correct/len(Actual_Fatalities))